You are given an **undirected weighted connected** graph containing `n` nodes labeled from `0` to `n - 1`, and an integer array `edges` where **edges[i] = [a<sub>i</sub>, b<sub>i</sub>, w<sub>i</sub>]** indicates that there is an edge between nodes **a<sub>i</sub>** and **b<sub>i</sub>** with weight **w<sub>i</sub>**.

Some edges have a weight of `-1` (**w<sub>i</sub> = -1**), while others have a **positive** weight (**w<sub>i</sub> > 0**).

Your task is to modify **all edges** with a weight of `-1` by assigning them **positive integer values** in the range **[1, 2 * 10<sup>9</sup>]** so that the **shortest distance** between the nodes `source` and `destination` becomes equal to an integer `target`. If there are **multiple modifications** that make the shortest distance between `source` and `destination` equal to `target`, any of them will be considered correct.

Return *an array containing all edges (even unmodified ones) in any order if it is possible to make the shortest distance from* `source` *to* `destination` *equal to* `target`, *or an ***empty array*** if it's impossible*.

**Note:** You are not allowed to modify the weights of edges with initial positive weights.

<br>

**Example 1:**

![graph-1](../../images/graph-1.png)

>**Input:** n = 5, edges = [[4,1,-1],[2,0,-1],[0,3,-1],[4,3,-1]], source = 0, destination = 1, target = 5<br>
>**Output:** [[4,1,1],[2,0,1],[0,3,3],[4,3,1]]<br>
>**Explanation:** The graph above shows a possible modification to the edges, making the distance from 0 to 1 equal to 5.

**Example 2:**

![graph-2](../../images/graph-2.png)

>**Input:** n = 3, edges = [[0,1,-1],[0,2,5]], source = 0, destination = 2, target = 6<br>
>**Output:** []<br>
>**Explanation:** The graph above contains the initial edges. It is not possible to make the distance from 0 to 2 equal to 6 by modifying the edge with weight -1. So, an empty array is returned.

**Example 3:**

![graph-3](../../images/graph-3.png)

>**Input:** n = 4, edges = [[1,0,4],[1,2,3],[2,3,5],[0,3,-1]], source = 0, destination = 2, target = 6<br>
>**Output:** [[1,0,4],[1,2,3],[2,3,5],[0,3,1]]<br>
>**Explanation:** The graph above shows a modified graph having the shortest distance from 0 to 2 as 6.

<br>

**Constraints:**
- >1 <= n <= 100
- >1 <= edges.length <= n * (n - 1) / 2
- >edges[i].length == 3
- >0 <= a<sub>i</sub>, b<sub>i</sub> < n
- >w<sub>i</sub> = -1 or 1 <= w<sub>i</sub> <= 10<sup>7</sup>
- >a<sub>i</sub> != b<sub>i</sub>
- >0 <= source, destination < n
- >source != destination
- >1 <= target <= 10<sup>9</sup>
- >The graph is connected, and there are no self-loops or repeated edges

In [1]:
import heapq

class Solution:
    def modifiedGraphEdges(self, n: int, edges: list[list[int]], source: int, destination: int, target: int) -> list[list[int]]:
        adjacency_list = [[] for _ in range(n)]
        
        for i, (nodeA, nodeB, weight) in enumerate(edges):
            adjacency_list[nodeA].append((nodeB, i))
            adjacency_list[nodeB].append((nodeA, i))

        distances = [[float('inf')] * 2 for _ in range(n)]
        distances[source][0] = distances[source][1] = 0

        self.dijkstra(adjacency_list, edges, distances, source, 0, 0)
        difference = target - distances[destination][0]

        if difference < 0:
            return []

        self.dijkstra(adjacency_list, edges, distances, source, difference, 1)

        if distances[destination][1] < target:
            return []

        for edge in edges:
            if edge[2] == -1:
                edge[2] = 1

        return edges

    def dijkstra(self, adjacency_list, edges, distances, source, difference, run):
        n = len(adjacency_list)
        priority_queue = [(0, source)]
        distances[source][run] = 0

        while priority_queue:
            current_distance, current_node = heapq.heappop(priority_queue)
            if current_distance > distances[current_node][run]:
                continue

            for next_node, edge_index in adjacency_list[current_node]:
                weight = edges[edge_index][2]
                if weight == -1:
                    weight = 1
                
                if run == 1 and edges[edge_index][2] == -1:
                    new_weight = difference + distances[next_node][0] - distances[current_node][1]
                    if new_weight > weight:
                        edges[edge_index][2] = weight = new_weight

                if distances[next_node][run] > distances[current_node][run] + weight:
                    distances[next_node][run] = distances[current_node][run] + weight
                    heapq.heappush(priority_queue, (distances[next_node][run], next_node))